In [1]:
import cv2                 


In [2]:
import numpy as np         
import os                  
from random import shuffle 
from tqdm import tqdm   

TRAIN_DIR = 'C:/Users/Bharath/Desktop/datasets/dogs-vs-cats/train'
TEST_DIR = 'C:/Users/Bharath/Desktop/datasets/dogs-vs-cats/test'
IMG_SIZE = 50
LR = 1e-3

MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR, '2conv-basic')

In [3]:
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat': return [1,0]
    elif word_label == 'dog': return [0,1]

In [4]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [5]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data

In [6]:
train_data = create_train_data()
# If you have already created the dataset:
#train_data = np.load('train_data.npy')

100%|██████████| 17401/17401 [02:33<00:00, 113.15it/s]


In [16]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

import tensorflow as tf
tf.reset_default_graph()


# Building convolutional convnet
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')
# http://tflearn.org/layers/conv/
# http://tflearn.org/activations/
convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

#convnet = conv_2d(convnet, 32, 2, activation='relu')
#convnet = max_pool_2d(convnet, 2)

#convnet = conv_2d(convnet, 64, 2, activation='relu')
#convnet = max_pool_2d(convnet, 2)

#convnet = conv_2d(convnet, 32, 2, activation='relu')
#convnet = max_pool_2d(convnet, 2)

#convnet = conv_2d(convnet, 64, 2, activation='relu')
#convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir = 'log')


if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

train = train_data[:-500]
test = train_data[-500:]

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

model.fit({'input': X}, {'targets': Y}, n_epoch=1, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)


Training Step: 529  | total loss: 6.29756 | time: 149.593s
| Adam | epoch: 001 | loss: 6.29756 - acc: 0.7265 -- iter: 16896/16901
Training Step: 530  | total loss: 6.17150 | time: 150.745s
| Adam | epoch: 001 | loss: 6.17150 - acc: 0.7320 | val_loss: 6.95381 - val_acc: 0.6980 -- iter: 16901/16901
--


In [17]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loded!')

INFO:tensorflow:Restoring parameters from C:\Users\Bharath\Desktop\Neural Networks\dogsvscats-0.001-2conv-basic.model
model loded!


In [18]:
train = train_data[:-500]
test = train_data[-500:]

In [19]:
X = np.array([i[0]for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1]for i in train]

test_x = np.array([i[0]for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1]for i in train]

## size matters

In [ ]:
# tensorboard --logdir=foo:C:\Users\Bharath\Desktop\jupyter\tensorTest\log

In [20]:
model.save(MODEL_NAME)

INFO:tensorflow:C:\Users\Bharath\Desktop\Neural Networks\dogsvscats-0.001-2conv-basic.model is not in all_model_checkpoint_paths. Manually adding it.


## increase the number of ReLU layers and epochs for perfection

In [22]:
import matplotlib.pyplot as plt

# if you need to create the data:
test_data = process_test_data()
# if you already have some saved:
#test_data = np.load('test_data.npy')

fig=plt.figure()

for num,data in enumerate(test_data[:12]):
    # cat: [1,0]
    # dog: [0,1]
    
    img_num = data[1]
    img_data = data[0]
    
    y = fig.add_subplot(3,4,num+1)
    orig = img_data
    data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
    #model_out = model.predict([data])[0]
    model_out = model.predict([data])[0]
    
    if np.argmax(model_out) == 1: str_label='Dog'
    else: str_label='Cat'
        
    y.imshow(orig,cmap='gray')
    plt.title(str_label)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()

100%|█████████▉| 4895/4901 [01:06<00:00, 73.27it/s]

error: OpenCV(3.4.1) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function cv::resize


In [ ]:
with open('submission_file.csv','w') as f:
    f.write('id,label\n')
            
with open('submission_file.csv','a') as f:
    for data in tqdm(test_data):
        img_num = data[1]
        img_data = data[0]
        orig = img_data
        data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
        model_out = model.predict([data])[0]
        f.write('{},{}\n'.format(img_num,model_out[1]))